`comparaison_env`

In [1]:
import pandas as pd
import glob
import os 
import sys
import trimesh
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath("/home/pelissier/These-ATER/Papier_international3/Dataset"))
from utils import *

Variables globales

In [2]:
### MODELNET40 REMESHING ISO
ModelNet40_remeshing_iso_dir = "/home/pelissier/These-ATER/Papier_international3/Dataset/ModelNet40_remeshing_iso"
# Data des 12 caméras du Rendu
data_modelnet_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Dataset/Rendu/ModelNet40/circular_config_12_elevation_30_R2.csv")
# Path ok (avec limper + projection + bvs)
paths_bvs = read_paths_from_txt("/home/pelissier/These-ATER/Papier_international3/Dataset/paths_files/obj_SMPLER_files_ModelNet40_remeshing_iso_user-study-844-ok.txt"); print(len(paths_bvs))

##################################################################################################################################
### User study
# Path to the directory containing the csv files of the user study
us_dir = "/home/pelissier/These-ATER/Papier_internationale2/Validation/user_study/3D/post_traitement/csv_etude_Prolific"
# Paths of the 44 csv file containing the results of the user study
paths_us_csv = glob.glob(os.path.join(us_dir, "**", "*tache_normalise.csv*"), recursive=True); print(len(paths_us_csv))
# Data of camera poses in user study : i, j, theta, delta, X, Y, Z
data_us_cam = pd.read_csv("/home/pelissier/These-ATER/Papier_international3/Code/Comparaison-User-study/cam_pose_rep_etude.csv")
# Path data folder of user study
Data_dir = "/home/pelissier/These-ATER/Papier_internationale2/Data"
# Path of user-study outputs folder in Dataset
user_study_dir = "/home/pelissier/These-ATER/Papier_international3/Dataset/user_study"

##################################################################################################################################
# Correspondances entre les noms des modèles dans ModelNet10 et les noms des modèles dans l'User Study
match_ModelNet2US = {'aeroplane': 'A380', "chair":'chair107', 'bench': 'bench', 'dresser': 'cabinet', 'night_stand': 'cabinet', 'vase': 'vase', 'cup':'cup', 'car': 'carVasa'}


844
44


### Fonctions

In [ ]:
def rotation(theta, u):
    c = np.cos(np.deg2rad(theta))
    s = np.sin(np.deg2rad(theta))
    ux = u[0]; uy=u[1]; uz=u[2]

    R = np.array(
        [[ux*ux*(1-c)+c, ux*uy*(1-c)-uz*s, ux*uz*(1-c)+uy*s],
        [ux*uy*(1-c)+uz*s, uy*uy*(1-c)+c, uy*uz*(1-c)-ux*s],
        [ux*uz*(1-c)-uy*s, uy*uz*(1-c)+ux*s, uz*uz*(1-c)+c]])
    return R

### Choix des utilisateurs --> couronne 'Milieu-Dessus' + ordre en fonction des choix

In [ ]:
results = {}

# Les POV qui nous intéressent sont ceux sur la couronne 'Milieu-Dessus', donc avec j == 1
for path_us_csv in paths_us_csv:
    df = pd.read_csv(path_us_csv)
    # name of model
    name = df['mesh_name'][0]
    # Choix des utilisateurs
    results[name] = {}
    results[name]['choix_1'] = [(row['pose_i'], row['pose_j']) for _, row in df.iterrows() if ((row['pose_j'] == 1) and (row['num_choix'] == 1))]      
    results[name]['choix_2'] = [(row['pose_i'], row['pose_j']) for _, row in df.iterrows() if ((row['pose_j'] == 1) and (row['num_choix'] == 2))]      
    results[name]['choix_3'] = [(row['pose_i'], row['pose_j']) for _, row in df.iterrows() if ((row['pose_j'] == 1) and (row['num_choix'] == 3))]      

In [ ]:
results["carVasa"]

### Mise en correspondance cameras

In [ ]:
paths_bvs_car = [paths_bvs[i] for i in range(len(paths_bvs)) if "car" in paths_bvs[i]]; print(len(paths_bvs_car))
paths_bvs = paths_bvs_car

In [ ]:
# Coordonnées des 8 caméras de l'étude utilateur
X_us = np.array(data_us_cam.loc[data_us_cam['j'] == 1]['X_rep_etude'])
Y_us = np.array(data_us_cam.loc[data_us_cam['j'] == 1]['Y_rep_etude'])
Z_us = np.array(data_us_cam.loc[data_us_cam['j'] == 1]['Z_rep_etude'])

# Coordonnées des 12 caméras de ModelNet40
X_modelnet = np.array(data_modelnet_cam['LocationX'][1:])
Y_modelnet = np.array(data_modelnet_cam['LocationY'][1:])
Z_modelnet = np.array(data_modelnet_cam['LocationZ'][1:])

## Model Random
# mesh random from ModelNet40 (remeshing_iso)
path_mesh = paths_bvs[0]; print(path_mesh)
categorie = path_mesh.split("/")[-3]
type = path_mesh.split("/")[-2]
name = os.path.basename(path_mesh).split("_")[0]
path_mesh_modelnet = os.path.join(ModelNet40_remeshing_iso_dir, paths_bvs[0]+".obj"); print(path_mesh_modelnet)
mesh_modelnet = trimesh.load_mesh(path_mesh_modelnet)

# mesh random from User_study
mesh_name_us = match_ModelNet2US[categorie]; print(mesh_name_us)
path_mesh_us = os.path.join(Data_dir, mesh_name_us, mesh_name_us+"_update_normed_centered_user_study.obj"); print(path_mesh_us)
mesh_us = trimesh.load_mesh(path_mesh_us)

In [ ]:
## OBJ file with (12 cameras + obj)
# Vertices and faces of the model user study
verts_mesh_modelnet = np.array(mesh_modelnet.vertices)
faces_mesh_modelnet = np.array(mesh_modelnet.faces) 
# colors : Blanc cam 1 --> Jaune cam 4 ---> Rouge cam 10 --> Rouge foncé cam 12
colormap = plt.get_cmap('hot'); colors_modelnet = colormap(np.linspace(0, 1, 12))[::-1] 
# Write obj
with open(os.path.join("/home/pelissier/These-ATER/Papier_international3/Dataset/tmp", name+"_modelNet40+12cam.obj"), 'w') as obj_file:
    # Write vertices
    for vertex in verts_mesh_modelnet:
        obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]} 128 128 128\n")      
    # Write camera positions
    for i in range(12):
        r, g, b, _ = colors_modelnet[i]; print(r, g, b)
        obj_file.write(f"v {X_modelnet[i]} {Y_modelnet[i]} {Z_modelnet[i]} {r*255} {g*255} {b*255}\n")
    # Write faces
    for face in faces_mesh_modelnet:
        # Convert face indices to 1-based indexing
        obj_file.write(f"f {' '.join(str(idx + 1) for idx in face)}\n")   
 
## OBJ file with (8 cameras + obj)
# Vertices and faces of the model user study
verts_mesh_us = np.array(mesh_us.vertices)
faces_mesh_us = np.array(mesh_us.faces) 
# colors : Blanc cam 1 --> Jaune cam 4 ---> Rouge cam 10 --> Rouge foncé cam 12
colormap = plt.get_cmap('hot'); colors_us = colormap(np.linspace(0, 1, 8))[::-1] 
# Write obj
with open(os.path.join("/home/pelissier/These-ATER/Papier_international3/Dataset/tmp", name+"_us+8cam.obj"), 'w') as obj_file:
    # Write vertices
    for vertex in verts_mesh_us:
        obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]} 128 128 128\n")      
    # Write camera positions
    for i in range(8):
        r, g, b, _ = colors_us[i]; print(r, g, b)
        obj_file.write(f"v {X_us[i]} {Y_us[i]} {Z_us[i]} {r*255} {g*255} {b*255}\n")
    # Write faces
    for face in faces_mesh_us:
        # Convert face indices to 1-based indexing
        obj_file.write(f"f {' '.join(str(idx + 1) for idx in face)}\n")   

### Match US et ModelNet40

In [ ]:
def US2ModelNet40(vertices):
    angle = 90; axe_X = [1,0,0]
    Rx = rotation(angle, axe_X)
    angle = -90; axe_Z = [0,0,1]
    Rz = rotation(angle, axe_Z)
    return np.around(np.dot(Rz, np.dot(Rx, vertices)),3)

print(US2ModelNet40(np.array([1,0,0])) == [0,-1,0])
print(US2ModelNet40(np.array([0,1,0])) == [0,0,1])
print(US2ModelNet40(np.array([0,0,1])) == [-1,0,0])

In [ ]:
# Load us study data
for mesh_categorie in match_ModelNet2US.keys():
    #mesh_categorie = 'car'
    ## User study
    # Load model from User study
    mesh_name_us = match_ModelNet2US[mesh_categorie]
    mesh_us = trimesh.load_mesh(os.path.join(Data_dir, mesh_name_us, mesh_name_us+"_update_normed_centered_user_study.obj"))
    # Vertices and faces of the model user study
    verts_mesh_us = np.array(mesh_us.vertices)
    faces_mesh_us = np.array(mesh_us.faces) 
    # Changement de repère
    new_verts_mesh_us = np.transpose(US2ModelNet40(np.transpose(verts_mesh_us)))
    # Write obj
    with open(os.path.join(user_study_dir, mesh_name_us+"_modelNet40.obj"), 'w') as obj_file:
        # Write vertices
        for vertex in new_verts_mesh_us:
            obj_file.write(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")

        # Write faces
        for face in faces_mesh_us:
            # Convert face indices to 1-based indexing
            obj_file.write(f"f {' '.join(str(idx + 1) for idx in face)}\n")   
            
    ## Number of mesh of categorie ok
    paths_mesh = [path for path in paths_bvs if mesh_categorie in path]; print(len(paths_mesh))
       

In [ ]:
paths_mesh

In [ ]:
print(np.around(Objectnet2Etude([0,-1,0])[0],3) == [1,0,0])